# Import dependencies

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import os,sys

# Keypoints using MP Holistic

### Definitions

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image,results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [5]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image,results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(43,180,255), thickness = 1,circle_radius=2),
                             mp_drawing.DrawingSpec(color=(255,0,255), thickness = 1,circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(43,180,255), thickness = 2,circle_radius=3),
                             mp_drawing.DrawingSpec(color=(255,0,255), thickness = 3,circle_radius=3)
                             )
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(43,180,255), thickness = 2,circle_radius=3),
                             mp_drawing.DrawingSpec(color=(255,0,255), thickness = 2,circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(43,180,255), thickness = 2,circle_radius=3),
                             mp_drawing.DrawingSpec(color=(255,0,255), thickness = 2,circle_radius=1)
                             )

In [6]:
def draw_styled_landmarks_np(image, results):
    mp_drawing.draw_landmarks(image,results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(43,180,255), thickness = 1,circle_radius=2),
                             mp_drawing.DrawingSpec(color=(255,0,255), thickness = 1,circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(43,180,255), thickness = 2,circle_radius=3),
                             mp_drawing.DrawingSpec(color=(255,0,255), thickness = 2,circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(43,180,255), thickness = 2,circle_radius=3),
                             mp_drawing.DrawingSpec(color=(255,0,255), thickness = 2,circle_radius=1)
                             )

### Detecting from an image

In [10]:
#Enter picture input/ file path
rawpic = os.path.join("Guardian Demon 18.png")

#convert to a numpy array
pic = cv2.imread(rawpic)

In [15]:
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    #make detections
    picpr, results = mediapipe_detection(pic, holistic)
    
    #draw landmarks
    #draw_landmarks(image, results)
    draw_styled_landmarks(picpr, results)
    
    #display image
    plt.imshow(cv2.cvtColor(picpr, cv2.COLOR_BGR2RGB))
    plt.show()

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [9]:
#Save output as jpeg
cv2.imwrite("out.jpg", picpr)

True

### Detecting from Webcam

#### Overlayed

In [7]:
cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        #read frame
        ret, frame = cap.read()

        #make detections
        image, results = mediapipe_detection(frame, holistic)
               
        #draw landmarks
        #draw_landmarks(image, results)
        draw_styled_landmarks(image, results)

        #show to screen
        cv2.imshow('OpenCV Feed', image)
        
        #break gracefully
        if cv2.waitKey(10) & 0xFF ==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

#### Black background

In [8]:
#First large number is the number of pixels in the columns or width
frame1 = np.full((640,3),0)
# the *160 is the number of pixels in the rows/ height divided by 3
frame2 = np.array([frame1,frame1,frame1]*160,dtype=np.uint8)

cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        #read frame
        ret, frame = cap.read()

        #make detections
        image, results = mediapipe_detection(frame, holistic)
        
        #style stuff, change frame2 to image from draw landmarks below for overlayed output
        frame2 = np.array([frame1,frame1,frame1]*160,dtype=np.uint8)
               
        #draw landmarks
        #draw_landmarks(image, results)
        draw_styled_landmarks(frame2, results)

        #show to screen
        cv2.imshow('OpenCV Feed', frame2)
        
        #break gracefully
        if cv2.waitKey(10) & 0xFF ==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

### Detecting from a video

In [41]:
#Enter video input/ file path
video = "signtest.avi"

#### Overlayed

In [15]:
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    #Establish capture
    cap = cv2.VideoCapture(os.path.join(video))
    
    #Setup Video writer
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    #Videowriter
    video_writer = cv2.VideoWriter(os.path.join('output3.mp4'), cv2.VideoWriter_fourcc('M','P','4','2'), fps, (width, height))
    
    # Loop through each frame
    for frame_idx in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
        #read frame
        ret, frame = cap.read()

        #make detections
        image, results = mediapipe_detection(frame, holistic)

        #draw landmarks
        #draw_landmarks(image, results)
        draw_styled_landmarks(image, results)

        #show to screen
        cv2.imshow('OpenCV Feed', image)
        
        #Write out frame
        #video_writer.write(image)

        #Breaking the loop
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    #close down everything
    cap.release()
    cv2.destroyAllWindows()
    video_writer.release()

#### Black Background

In [16]:
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    #Establish capture
    cap = cv2.VideoCapture(os.path.join(video))
    
    #Setup Video writer
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    frame1 = np.full((width,3),0)
        
    #Videowriter
    video_writer = cv2.VideoWriter(os.path.join('signttestOutput.mp4'), cv2.VideoWriter_fourcc('P','I','M','1'), fps, (width, height))
    
    # Loop through each frame
    for frame_idx in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
        #read frame
        ret, frame = cap.read()

        #make detections
        image, results = mediapipe_detection(frame, holistic)
        
        frame2 = np.array([frame1,frame1,frame1]*int(height/3),dtype=np.uint8)

        #draw landmarks
        #draw_landmarks(image, results)
        draw_styled_landmarks(frame2, results)
        #draw_styled_landmarks_np(frame2, results)

        #show to screen
        cv2.imshow('OpenCV Feed', frame2)
        
        #Write out frame
        #video_writer.write(frame2)

        #Breaking the loop
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    #close down everything
    cap.release()
    cv2.destroyAllWindows()
    video_writer.release()

## Extracting keypoint values

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh =np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

## Setup Folders for Collection

In [8]:
#Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_DATA')

#Actions that we are trying to deetect
#actions = np.array([ ' A', ' B', ' C', ' D', ' E', ' F', ' G', ' H', ' I', ' J', ' K', ' L', ' M', ' N', ' O', ' P', ' Q', ' R', ' S', ' T', ' U', ' V', ' W', ' X', ' Y', ' Z', 'Hello', 'my', 'name'])
actions = np.array([ 'Hello', 'my', 'name'])

#Thirty videos worth of data
no_sequences = 30

#videos are going to be 30 frames in length
sequence_length = 30

In [9]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
                pass

## Collect Keypoint Values for Training and Testing

In [44]:
cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    #loop through actions
    for action in actions:
        #loop through videos
        for sequence in range(no_sequences):
            #loop through video length aka sequence length
            for frame_num in range(sequence_length):
                
                #read frame
                ret, frame = cap.read()

                #make detections
                image, results = mediapipe_detection(frame, holistic)

                #draw landmarks
                #draw_landmarks(image, results)
                draw_styled_landmarks(image, results)
                
                #apply wait logic
                if frame_num == 0:
                    cv2.putText(image,'STARTING COLLECTION',(120,200),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4,cv2.LINE_AA)
                    cv2.waitKey(2000)
                    cv2.putText(image,'Collecting frames for {} Video Number{}'.format(action, sequence),(15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1,cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image,'Collecting frames for {} Video Number{}'.format(action, sequence),(15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1,cv2.LINE_AA)
                    
                #Emport keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence),str(frame_num))
                np.save(npy_path, keypoints)

                #show to screen
                cv2.imshow('OpenCV Feed', image)

                #break gracefully
                if cv2.waitKey(10) & 0xFF ==ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

KeyboardInterrupt: 

## Preprocess Data and Create Labels and Features

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [11]:
label_map = {label:num for num, label in enumerate(actions)}

In [12]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [13]:
X = np.array(sequences)

In [14]:
y = to_categorical(labels).astype(int)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

## Build and Train LSTM Neural Network

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [36]:
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir)

In [37]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [38]:
model.compile(optimizer = 'Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [20]:
model.fit(X_train, y_train, epochs = 2000, callbacks = [tb_callback])

Epoch 1/2000
3/3 [==============================] - 6s 146ms/step - loss: 1.3249 - categorical_accuracy: 0.3176
Epoch 2/2000
3/3 [==============================] - 0s 129ms/step - loss: 1.1833 - categorical_accuracy: 0.3647
Epoch 3/2000
3/3 [==============================] - 0s 154ms/step - loss: 2.1442 - categorical_accuracy: 0.4941
Epoch 4/2000
3/3 [==============================] - 0s 140ms/step - loss: 0.8579 - categorical_accuracy: 0.6353
Epoch 5/2000
3/3 [==============================] - 1s 211ms/step - loss: 2.6473 - categorical_accuracy: 0.4235
Epoch 6/2000
3/3 [==============================] - 1s 210ms/step - loss: 0.9729 - categorical_accuracy: 0.5294
Epoch 7/2000
3/3 [==============================] - 0s 150ms/step - loss: 1.0072 - categorical_accuracy: 0.3529
Epoch 8/2000
3/3 [==============================] - 0s 151ms/step - loss: 0.9702 - categorical_accuracy: 0.3647
Epoch 9/2000
3/3 [==============================] - 0s 163ms/step - loss: 0.8939 - categorical_accuracy:

KeyboardInterrupt: 

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-trai

## Make predictons

In [22]:
res = model.predict(X_test)

In [23]:
actions[np.argmax(res[2])]

'Hello'

In [24]:
actions[np.argmax(y_test[2])]

'Hello'

## Save Weights

In [33]:
model.save('action3.h5')

In [34]:
del model

In [39]:
model.load_weights('action.h5')

## Evaluation using confusion Matrix and Accuracy

In [25]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [26]:
yhat = model.predict(X_test)

In [27]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat =np.argmax(yhat, axis=1).tolist()

In [28]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[2, 0],
        [0, 3]],

       [[3, 0],
        [0, 2]]], dtype=int64)

In [29]:
accuracy_score(ytrue, yhat)

1.0

## Testing the model

In [30]:
colors = [(245,117,16),(117,245,16),(16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100),90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0,85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

### From webcam

In [40]:
#New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.6

cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        #read frame
        ret, frame = cap.read()

        #make detections
        image, results = mediapipe_detection(frame, holistic)
               
        #draw landmarks
        #draw_landmarks(image, results)
        draw_styled_landmarks(image, results)
        
        #Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))

        #visualisation logic
            if np.unique(predictions[-10:])[0]==np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence)>0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])


                if len(sentence) > 5:
                    sentence = sentence[-5:]
                
                #Visualise probalilties
                image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640,40),(245,117,16), -1 )
        cv2.putText(image,  ' '.join(sentence), (3,30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        #show to screen
        cv2.imshow('OpenCV Feed', image)
        
        #break gracefully
        if cv2.waitKey(10) & 0xFF ==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
my
my
my
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
name
name
name
name
name
my
my
my
my
my
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name

### From video

In [42]:
#Enter video input/ file path
video = "signtest.avi"

In [43]:
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    #Establish capture
    cap = cv2.VideoCapture(os.path.join(video))
    
    #Setup Video writer
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    #Videowriter
    video_writer = cv2.VideoWriter(os.path.join('output3.mp4'), cv2.VideoWriter_fourcc('M','P','4','2'), fps, (width, height))
    
    # Loop through each frame
    for frame_idx in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
        #read frame
        ret, frame = cap.read()

        #make detections
        image, results = mediapipe_detection(frame, holistic)

        #draw landmarks
        #draw_landmarks(image, results)
        draw_styled_landmarks(image, results)
        
        #Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))

        #visualisation logic
            if np.unique(predictions[-10:])[0]==np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence)>0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])


                if len(sentence) > 5:
                    sentence = sentence[-5:]
                
                #Visualise probalilties
                image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640,40),(245,117,16), -1 )
        cv2.putText(image,  ' '.join(sentence), (3,30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

        #show to screen
        cv2.imshow('OpenCV Feed', image)
        
        #Write out frame
        #video_writer.write(image)

        #Breaking the loop
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    #close down everything
    cap.release()
    cv2.destroyAllWindows()
    video_writer.release()

name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
name
my
my
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
my
my
my
my
my
my
my
my
my
my
my
my
my
my
my
my
my
my
my
my
my
my
my
my
my
my
my
my
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
